###  Notebook to demonstrate Self-supervised Finetuning Workflow

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. Train Adapt Optimize (TAO) Toolkit is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

![image](https://d29g4g2dyqv443.cloudfront.net/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png)

### TAO Architecture

This notebook uses the **TAO SDK** for all API interactions. Key architectural features:


- **Job-Centric Architecture**: Operations are organized around jobs
- **SDK-Based**: Takes advantage of `TaoClient` SDK for faster development
- **Persistent Credentials**: Authentication uses `TAO_BASE_URL`, `TAO_ORG`, `TAO_TOKEN` environment variables or same in ~/.tao/config

### The workflow in a nutshell

- Set up TAO SDK authentication
- Pull datasets from cloud storage
- Select Pre-trained Model (PTM) from NGC
- **Job Actions**:
    - Train (Normal/AutoML)
    - Export
    - Inference
    - Delete jobs/datasets

### Table of contents

1. [FIXME's](#head-1)
1. [SDK Authentication](#head-2)
1. [Create a cloud workspace](#head-2)
1. [Set dataset formats](#head-3)
1. [Create and pull train dataset](#head-4)
1. [Create and pull test dataset](#head-5)
1. [List the created datasets](#head-6)
1. [List base experiments (PTMs)](#head-10)
1. [Assign datasets](#head-11)
1. [Assign PTM](#head-12)
1. [Prepare AutoML Configuration](#head-13)
1. [Create Training Job](#head-14)
1. [Monitor Job Status](#head-15)
1. [Create Export Job](#head-18)
1. [Create Inference Job](#head-20)
1. [Delete jobs](#head-22)
1. [Delete datasets](#head-23)

### Requirements
- TAO API Fine-Tuning Microservce (FTMS) deployment
- TAO SDK: `pip install nvidia-tao-client`
- NGC API key and organization access
- Please find the server requirements [here](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_setup.html#)

In [ ]:
import json
import os
import time
from IPython.display import clear_output

# Import TAO SDK
from tao_sdk.client import TaoClient

In [ ]:
# Restore variable in case of jupyter session restart and resume execution where it left off
%store -r model_name
%store -r automl_enabled
%store -r automl_algorithm
%store -r tao_client
%store -r workspace_id
%store -r train_dataset_id
%store -r test_dataset_id
%store -r job_map

### To see the dataset folder structure required for the models supported in this notebook, visit the notebooks under dataset_prepare like for [this notebook](../dataset_prepare/foundational_model_finetuning.ipynb)

### FIXME's <a class="anchor" id="head-1"></a>

1. Assign a model_name in FIXME 1
1. (Optional) Enable AutoML if needed in FIXME 2
1. (Optional) Choose between bayesian and hyperband automl_algorithm in FIXME 3 (If automl was enabled in FIXME2)
1. Set environment variables for TAO API in FIXME 4 (TAO_BASE_URL, TAO_ORG, TAO_TOKEN)
1. Assign the ngc_key variable in FIXME 5 (for login)
1. Assign the ngc_org_name variable in FIXME 6 (for login)
1. Set cloud storage details in FIXME 7
1. Assign path of datasets relative to the bucket in FIXME 8

#### Choose a SSL model

In [ ]:
# FIXME 1: Define model_name workspaces and other variables
# Available models:
# 1. nvdinov2 - https://docs.nvidia.com/tao/tao-toolkit/text/cv_finetuning/pytorch/nvdinov2.html

os.environ["TAO_MODEL_NAME"] = model_name = os.environ.get("TAO_MODEL_NAME", "nvdinov2")  # Pick the model name from the above mentioned list
%store model_name

#### Toggle AutoML params
[AutoML documentation](https://docs.nvidia.com/tao/tao-toolkit/text/automl/automl.html#getting-started)

In [ ]:
# FIXME 2: Set to True if you want to run automl for the model chosen in the previous cell
automl_enabled = os.environ.get("TAO_AUTOML_ENABLED", "False").lower() == "true"
os.environ["TAO_AUTOML_ENABLED"] = str(automl_enabled)
# One of bayesian/hyperband
os.environ["TAO_AUTOML_ALGORITHM"] = automl_algorithm = os.environ.get("TAO_AUTOML_ALGORITHM", "bayesian")

%store automl_enabled
%store automl_algorithm

#### Set TAO API Environment Variables

In [ ]:
# FIXME 4: Set TAO API environment variables

# Set to your TAO API endpoint
os.environ["TAO_BASE_URL"] = os.environ.get("TAO_BASE_URL", "https://your_tao_ip_address:port/api/v2")

# These will be set after login
os.environ["TAO_ORG"] = os.environ.get("TAO_ORG", "your_ngc_org_name")
os.environ["TAO_TOKEN"] = os.environ.get("TAO_TOKEN", "your_token" )


#### Set NGC Personal key for authentication and NGC org to access API services

In [ ]:
# FIXME 5: Your NGC personal key
os.environ["NGC_KEY"] = ngc_key = os.environ.get("NGC_KEY", "your_ngc_key")

In [ ]:
# FIXME 6: Your NGC ORG name
os.environ["NGC_ORG"] = ngc_org_name = os.environ.get("NGC_ORG", "nvstaging")

### Login <a class="anchor" id="head-2"></a>

In [ ]:
# Initialize TAO Client and login using SDK
tao_client = TaoClient()

# Login using TAO SDK - this will automatically save credentials to environment variables
login_response = tao_client.login(
    ngc_key=ngc_key,
    ngc_org_name=ngc_org_name,
    enable_telemetry=True
)

print(" Login successful!")
print("JWT Token:", tao_client.token)
print("API Base URL:", tao_client.base_url)
print("Organization:", tao_client.org_name)

%store tao_client

### Get GPU details <a class="anchor" id="head-2"></a>

 One of the keys of the response json are to be used as platform_id when you run each job

In [ ]:
# # Get available GPU types using TAO SDK
# try:
#     gpu_types = tao_client.get_gpu_types()
#     print(" Available GPU types:")
#     print(json.dumps(gpu_types, indent=4))
# except Exception as e:
#     print(" Could not fetch GPU types (may not be available on this deployment):", str(e))

### Create cloud workspace
This workspace will be the place where your datasets reside and your results of TAO API jobs will be pushed to.

If you want to have different workspaces for dataset and experiment, duplocate the workspace creation part and adjust the metadata accordingly.

In [ ]:
# FIXME 7: Dataset Cloud bucket details to download dataset or push job artifacts for jobs

cloud_metadata = {}

# A Representative name for this cloud info
os.environ["TAO_WORKSPACE_NAME"] = cloud_metadata["name"] = os.environ.get("TAO_WORKSPACE_NAME", "AWS workspace info")

# Cloud specific details. Below is assuming AWS.
cloud_metadata["cloud_specific_details"] = {}

 # Whether it is AWS, HuggingFace or Azure
os.environ["TAO_WORKSPACE_CLOUD_TYPE"] = cloud_metadata["cloud_specific_details"]["cloud_type"] = os.environ.get("TAO_WORKSPACE_CLOUD_TYPE", "aws")

# Bucket region
os.environ["TAO_WORKSPACE_CLOUD_REGION"] = cloud_metadata["cloud_specific_details"]["cloud_region"] = os.environ.get("TAO_WORKSPACE_CLOUD_REGION", "us-west-1")

# Bucket name
os.environ["TAO_WORKSPACE_CLOUD_BUCKET_NAME"] = cloud_metadata["cloud_specific_details"]["cloud_bucket_name"] = os.environ.get("TAO_WORKSPACE_CLOUD_BUCKET_NAME", "bucket_name")

# Access and Secret keys
os.environ["TAO_WORKSPACE_CLOUD_ACCESS_KEY"] = cloud_metadata["cloud_specific_details"]["access_key"] = os.environ.get("TAO_WORKSPACE_CLOUD_ACCESS_KEY", "access_key")
os.environ["TAO_WORKSPACE_CLOUD_SECRET_KEY"] = cloud_metadata["cloud_specific_details"]["secret_key"] = os.environ.get("TAO_WORKSPACE_CLOUD_SECRET_KEY", "secret_key")

In [ ]:
# Create cloud workspace using TAO SDK
workspace_id = tao_client.create_workspace(
    name=cloud_metadata["name"],
    cloud_type=cloud_metadata["cloud_specific_details"]["cloud_type"],
    cloud_specific_details=cloud_metadata["cloud_specific_details"]
)

print("Workspace created successfully!")
print(f"Workspace ID: {workspace_id}")

# Get workspace details to confirm creation
workspace_details = tao_client.get_workspace_metadata(workspace_id)
print("Workspace details:")
print(json.dumps(workspace_details, indent=4))

%store workspace_id

#### Restore Database from backup file
#### This is optional and can be skipped if you don't want to restore the database from backup file

In [ ]:
# # Restore workspace from backup using TAO SDK
# try:    
#     # Construct workspace metadata that matches WorkspaceReq schema requirements
#     restore_workspace_metadata = {
#         "cloud_type": cloud_metadata["cloud_specific_details"]["cloud_type"],
#         "cloud_specific_details": cloud_metadata["cloud_specific_details"]
#     }
        
#     restore_result = tao_client.restore_workspace(
#         backup_file_name="mongodb_backup.gz",
#         workspace_metadata=restore_workspace_metadata
#     )
#     print(" Workspace restored successfully!")
#     print(json.dumps(restore_result, indent=4))
# except Exception as e:
#     print(" Workspace restore failed:", str(e))

#### Set dataset path (path within cloud bucket)

In [ ]:
# FIXME 8 : Set paths relative to cloud bucket
os.environ["TAO_TRAIN_DATASET_PATH"] = train_dataset_path =  os.environ.get("TAO_TRAIN_DATASET_PATH", f"/data/classification_train")
os.environ["TAO_TEST_DATASET_PATH"] = test_dataset_path = os.environ.get("TAO_TEST_DATASET_PATH", f"/data/classification_test")

### Set dataset formats <a class="anchor" id="head-3"></a>

In [ ]:
# Create train dataset
ds_type = "image_classification"
ds_format = "ssl"

### Create and pull train dataset <a class="anchor" id="head-4"></a>

In [ ]:
# Create train dataset using TAO SDK
train_dataset_id = tao_client.create_dataset(
    dataset_type=ds_type,
    dataset_format=ds_format,
    workspace_id=workspace_id,
    cloud_file_path=train_dataset_path,
    use_for=["training"]
)

print(" Train dataset created successfully!")
print(f"Train Dataset ID: {train_dataset_id}")

%store train_dataset_id

In [ ]:
# Check train dataset progress using TAO SDK
while True:
    clear_output(wait=True)
    dataset_details = tao_client.get_dataset_metadata(train_dataset_id)
    
    print(f" Train Dataset Status: {dataset_details.get('status', 'Unknown')}")
    print(f"Dataset ID: {train_dataset_id}")
    
    if dataset_details.get("status") == "invalid_pull":
        print("Dataset pull failed!")
        validation_details = dataset_details.get("validation_details", {})
        if validation_details:
            print("Validation details:")
            print(json.dumps(validation_details, indent=4))
        raise ValueError("Dataset pull failed")
        
    if dataset_details.get("status") == "pull_complete":
        print("Train dataset pull completed successfully!")
        print("Dataset details:")
        print(json.dumps(dataset_details, indent=4))
        break
        
    time.sleep(5)

In [ ]:
# # Update dataset path if needed using TAO SDK
# # Note: Dataset updates may require re-creation depending on the implementation
# try:
#     updated_dataset = tao_client.patch_dataset_metadata(train_dataset_id, {"cloud_file_path": "/data/classification_train_1"})
#     print(" Dataset updated successfully!")
#     print(json.dumps(updated_dataset, indent=4))
# except Exception as e:
#     print(" Dataset update not supported or failed:", str(e))

In [ ]:
# # Check dataset progress using TAO SDK (alternative approach)
# # This is an alternative way to monitor dataset status if needed

# while True:
#     clear_output(wait=True)
#     
#     try:
#         dataset_details = tao_client.get_dataset_metadata(train_dataset_id)
#         
#         print(f" Train Dataset Status: {dataset_details.get('status', 'Unknown')}")
#         print(f"Dataset ID: {train_dataset_id}")
#         print("Full dataset details:")
#         print(json.dumps(dataset_details, indent=4))
#         
#         if dataset_details.get("status") == "invalid_pull":
#             print(" Dataset pull failed!")
#             validation_details = dataset_details.get("validation_details", {})
#             if validation_details:
#                 print("Validation details:")
#                 print(json.dumps(validation_details, indent=4))
#             raise ValueError("Dataset pull failed")
#             
#         if dataset_details.get("status") == "pull_complete":
#             print(" Dataset pull completed successfully!")
#             break
#             
#     except Exception as e:
#         print(f" Error fetching dataset status: {str(e)}")
#         
#     time.sleep(5)

#### Uncomment if you want to remove corrupted images in your dataset

In [ ]:
# # This packages data-services experiments create and running the job of removing corrupted images
# from remove_corrupted_images import remove_corrupted_images_workflow
# # try:
#     from remove_corrupted_images import remove_corrupted_images_workflow
#     train_dataset_id = remove_corrupted_images_workflow(base_url, headers, workspace_id, train_dataset_id)
#     %store train_dataset_id
# except Exception as e:
#     raise e

### Create and pull test dataset <a class="anchor" id="head-5"></a>

In [ ]:
# Create test dataset using TAO SDK
test_dataset_id = tao_client.create_dataset(
    dataset_type=ds_type,
    dataset_format=ds_format,
    workspace_id=workspace_id,
    cloud_file_path=test_dataset_path,
    use_for=["testing"]
)

print("Test dataset created successfully!")
print(f"Test Dataset ID: {test_dataset_id}")

%store test_dataset_id

In [ ]:
# Check test dataset progress using TAO SDK
while True:
    clear_output(wait=True)
    dataset_details = tao_client.get_dataset_metadata(test_dataset_id)
    
    print(f" Test Dataset Status: {dataset_details.get('status', 'Unknown')}")
    print(f"Dataset ID: {test_dataset_id}")
    
    if dataset_details.get("status") == "invalid_pull":
        print(" Dataset pull failed!")
        validation_details = dataset_details.get("validation_details", {})
        if validation_details:
            print("Validation details:")
            print(json.dumps(validation_details, indent=4))
        raise ValueError("Dataset pull failed")
        
    if dataset_details.get("status") == "pull_complete":
        print("Test dataset pull completed successfully!")
        print("Dataset details:")
        print(json.dumps(dataset_details, indent=4))
        break
        
    time.sleep(5)

#### Uncomment if you want to remove corrupted images in your dataset

In [ ]:
# # This packages data-services experiments create and running the job of removing corrupted images
# from remove_corrupted_images import remove_corrupted_images_workflow
# # try:
#     from remove_corrupted_images import remove_corrupted_images_workflow
#     test_dataset_id = remove_corrupted_images_workflow(base_url, headers, workspace_id, test_dataset_id)
#     %store test_dataset_id
# except Exception as e:
#     raise e

### List the created datasets <a class="anchor" id="head-6"></a>

In [ ]:
# List datasets using TAO SDK
datasets = tao_client.list_datasets()

print(" Available datasets:")
print("id\t\t\t\t\t type\t\t\t format\t\t name")
print("-" * 100)

for dataset in datasets:
    dataset_id = dataset.get("id", "N/A")
    dataset_type = dataset.get("type", "N/A")
    dataset_format = dataset.get("format", "N/A")
    dataset_name = dataset.get("name", "N/A")
    print(f"{dataset_id}\t{dataset_type}\t{dataset_format}\t\t{dataset_name}")

In [ ]:
job_map = {}

### Experiment Details <a class="anchor" id="head-9"></a>

In [ ]:
# Experiment metadata
# These parameters will be used when creating the actual job:

encode_key = "nvidia_tao"
checkpoint_choose_method = "best_model"

# These will be used later in job creation
experiment_config = {
    "network_arch": model_name,
    "encryption_key": encode_key,
    "workspace": workspace_id
}

print(" Experiment configuration prepared for job creation:")
print(json.dumps(experiment_config, indent=4))

### List Base Experiments (PTMs) <a class="anchor" id="head-10"></a>

In [ ]:
# List base experiments (PTMs) using TAO SDK  
# These are the pre-trained models available for the selected network architecture
base_experiments = tao_client.list_base_experiments(filter_params={"network_arch": model_name})

print(f" Available base experiments (PTMs) for {model_name}:")
print("name\t\t\t     model id\t\t\t     network architecture")
print("-" * 120)

for exp in base_experiments:
    exp_name = exp.get("name", "N/A")
    exp_id = exp.get("id", "N/A")
    exp_arch = exp.get("network_arch", "N/A")
    print(f"{exp_name}\t{exp_id}\t{exp_arch}")

### Dataset Assignment <a class="anchor" id="head-11"></a>

### Assign PTM <a class="anchor" id="head-12"></a>

Search for the PTM on NGC for the SSL model chosen

In [ ]:
# List all pretrained models for the chosen network architecture using TAO SDK
base_experiments_detailed = tao_client.list_base_experiments(filter_params={"network_arch": model_name})

print(f" Available pre-trained models for {model_name}:")
print("-" * 100)

for exp in base_experiments_detailed:
    if "encryption_key" not in exp:  # Skip user-created experiments, show only PTMs
        ptm_name = exp.get("name", "N/A")
        ptm_version = exp.get("version", "N/A")
        ngc_path = exp.get("ngc_path", "N/A")
        print(f'PTM Name: {ptm_name}; PTM version: {ptm_version}; NGC PATH: {ngc_path}')

In [ ]:
# Assigning pretrained models to different networks
# From the output of previous cell make the appropriate changes to this map if you want to change the default PTM backbone.
# Changing the default backbone here requires changing default spec/config during train/infer etc like for example
# If you are changing the ptm to resnet34, then you have to modify the config key num_layers if it exists to 34 manually
pretrained_map = {"nvdinov2" : "nvaie/nv_dinov2_classification_model:trainable_v1.1",
                  }
no_ptm_models = set([])

In [ ]:
# Get pretrained model using TAO SDK
selected_ptm_id = None
if model_name not in no_ptm_models:
    base_experiments_detailed = tao_client.list_base_experiments(filter_params={"network_arch": model_name})
    
    # Search for PTM with given NGC path
    for exp in base_experiments_detailed:
        ngc_path = exp.get("ngc_path", "")
        if ngc_path.endswith(pretrained_map[model_name]):
            selected_ptm_id = exp.get("id")
            print(" Selected PTM metadata:")
            print(json.dumps(exp, indent=4))
            break
    
    if not selected_ptm_id:
        print(f" PTM with NGC path ending in '{pretrained_map[model_name]}' not found!")

In [ ]:
#  TAO: PTM assignment happens during job creation
# The selected PTM ID will be used in the job creation step
if model_name not in no_ptm_models and selected_ptm_id:
    print(f" PTM ID {selected_ptm_id} will be used as base_experiment_id in job creation")
else:
    print(" No PTM will be used (training from scratch)")

### Actions <a class="anchor" id="head-14"></a>

For all actions:
1. Get default spec schema and derive the default values
2. Modify defaults if needed
3. Post spec dictionary to the service
4. Run model action
5. Monitor job using retrieve
6. Download results using job download (if needed)

### Train <a class="anchor" id="head-15"></a>

#### View hyperparameters that are enabled for AutoML by default <a class="anchor" id="head-13"></a>

In [ ]:
# Initialize automl_params (needed for AutoML configuration)
automl_params = {}

if automl_enabled:
    # Get default AutoML parameters using TAO SDK
    # This is retrieved using the base experiment ID (PTM)
    automl_params = tao_client.get_automl_defaults(network_arch=model_name, action="train")
    print("Default AutoML parameters:")
    print(json.dumps(automl_params, sort_keys=True, indent=4))
else:
    print("AutoML is disabled - automl_params initialized as empty dict")

#### Prepare AutoML Configuration <a class="anchor" id="head-15.1"></a>

Refer to these hyper-links to see the parameters supported by each network and add more parameters if necessary in addition to the default automl enabled parameters: 

[NVDinoV2](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/nvdinov2/nvdinov2%20-%20train.csv)

In [ ]:
#  TAO: Prepare AutoML configuration for job creation
automl_information = None

if automl_enabled:
    # Choose any metric that is present in the kpi dictionary present in the model's status.json. 
    # Example status.json for each model can be found in the respective section in NVIDIA TAO DOCS here: https://docs.nvidia.com/tao/tao-toolkit/text/model_zoo/cv_models/index.html
    metric = "kpi"

    # Refer to parameter list mentioned in the above links and add/remove any extra parameter in addition to the default enabled ones in automl_specs
    automl_information = {
        "automl_enabled": True,
        "automl_algorithm": automl_algorithm,
        "automl_max_recommendations": 20,  # Only for bayesian
        "automl_R": 27,  # Only for hyperband
        "automl_nu": 3,  # Only for hyperband
        "epoch_multiplier": 1,  # Only for hyperband
        # Warning: The parameters that are disabled are not tested by TAO, so there might be unexpected behaviour in overriding this
        "override_automl_disabled_params": False,
        "automl_hyperparameters": str(automl_params),
        "metric": metric
    }
    
    print(" AutoML configuration prepared for job creation:")
    print(json.dumps(automl_information, sort_keys=True, indent=4))
    print(" This will be included in the job_run_experiment call")
else:
    print(" AutoML is disabled - training will use standard approach")

In [ ]:
# Get default train specs using TAO SDK
train_spec_response = tao_client.get_job_schema(action="train", network_arch=model_name)
train_specs = train_spec_response.get("default", {})
print(" Default train specifications:")
print(json.dumps(train_specs, sort_keys=True, indent=4))

In [ ]:
# Override any of the parameters listed in the previous cell as required
train_specs["train"]["num_gpus"] = 1
train_specs["train"]["num_epochs"] = 10
train_specs["train"]["checkpoint_interval"] = 10

print("Training specs:")
print(json.dumps(train_specs, sort_keys=True, indent=4))

In [ ]:
# Create experiment job using SDK

job_name = f"{model_name}_training_job"

# Prepare job creation parameters
job_params = {
    "kind": "experiment",
    "name": job_name,
    "network_arch": model_name,
    "encryption_key": encode_key,
    "workspace": workspace_id,
    "action": "train",
    "specs": train_specs,  # Pass as dict, not JSON string
    "base_experiment_ids": [selected_ptm_id] if selected_ptm_id else None,
    "train_datasets": [train_dataset_id] if train_dataset_id else None,
    "eval_dataset": test_dataset_id,
    "inference_dataset": test_dataset_id,
    "calibration_dataset": train_dataset_id,
    "automl_settings": automl_information if automl_information else None,
    # "platform_id": "9af1aa90-8ea5-5a11-98d9-3879cd0da92c",  # Optional: Pick from get_gpu_types output
}

# Create experiment job using TAO SDK interface
job_id = tao_client.create_job(**job_params)

print(" Experiment job created successfully!")
print(f"Job ID: {job_id}")
print(f"Job Name: {job_name}")
print(f"Network Architecture: {model_name}")
print(f"Action: train")
if automl_information:
    print(f"AutoML: {automl_information.get('automl_algorithm', 'N/A')} algorithm")

job_map["train_" + model_name] = job_id
print("\nJob Map:")
print(json.dumps(job_map, indent=4))
%store job_map

In [ ]:
# Monitor job status using TAO SDK
# For automl: Training times for different models benchmarked on 1 GPU V100 machine can be found here: https://docs.nvidia.com/tao/tao-toolkit/text/automl/automl.html#results-of-automl-experiments

train_job_id = job_map["train_" + model_name]

while True:
    clear_output(wait=True)
    
    try:
        job_status = tao_client.get_job_metadata(train_job_id)
        
        print(f"Training Job Status")
        print(f"Job ID: {train_job_id}")
        print(f"Status: {job_status.get('status', 'Unknown')}")
        print(f"Progress: {job_status.get('progress', 'N/A')}")
        
        # Show detailed status information
        print("\nDetailed Status:")
        print(json.dumps(job_status.get("job_details", {}), sort_keys=True, indent=4))
        
        current_status = job_status.get("status", "Unknown")
        
        if current_status == "Error":
            print(" Job failed!")
            break
            
        if current_status in ["Done", "Completed"]:
            print(" Job completed successfully!")
            break
            
        if current_status in ["Canceled", "Paused"]:
            print(f" Job {current_status}")
            break
            
    except Exception as e:
        print(f" Error fetching job status: {str(e)}")
        print("Job might still be starting up...")
        
    time.sleep(15)

In [ ]:
## To Stop an AutoML JOB
#    1. Stop the 'Monitor job status by repeatedly running this cell' cell (the cell right before this cell) manually
#    2. Uncomment the snippet in the next cell and run the cell

In [ ]:
# # Pause AutoML job using TAO SDK
# if automl_enabled:
#     train_job_id = job_map["train_" + model_name]
#     try:
#         pause_result = tao_client.pause_job(train_job_id)
#         print(" Job paused successfully!")
#         print(json.dumps(pause_result, indent=4))
#     except Exception as e:
#         print(f" Failed to pause job: {str(e)}")

In [ ]:
## Resume AutoML

In [ ]:
# # Resume AutoML job using TAO SDK
# # Uncomment the below snippet if you want to resume an already stopped AutoML job and then run the 'Monitor job status' cell above
# if automl_enabled:
#     train_job_id = job_map["train_" + model_name]
#     try:
#         resume_result = tao_client.resume_job(
#             job_id=train_job_id,
#             parent_job_id=None,
#             specs=json.dumps(train_specs)
#         )
#         print(" Job resumed successfully!")
#         print(json.dumps(resume_result, indent=4))
#     except Exception as e:
#         print(f" Failed to resume job: {str(e)}")

### Publish model

#### Edit the method of choosing checkpoint from list of train checkpoint files

In [ ]:
# Model handler parameters are managed differently
# Checkpoint selection is handled during job creation rather than experiment-level settings
# For now, we'll use the default checkpoint selection method
print(" In TAO, checkpoint selection is managed per-job rather than per-experiment")
print("Using default checkpoint selection method: best_model")

update_checkpoint_choosing = {
    "checkpoint_choose_method": "best_model",
    "checkpoint_epoch_number": {}
}
print("Current checkpoint choosing configuration:")
print(json.dumps(update_checkpoint_choosing, indent=4))

In [ ]:
# Checkpoint method configuration
# Checkpoint selection is handled per-job, not per-experiment
# You can configure this when creating export/inference jobs if needed

# Example: Change checkpoint selection method for future jobs
update_checkpoint_choosing["checkpoint_choose_method"] = "latest_model"  # Choose between best_model/latest_model/from_epoch_number
# Note: If from_epoch_number is chosen, you would specify the epoch in job creation specs

print("Checkpoint selection configuration updated:")
print(f"Method: {update_checkpoint_choosing['checkpoint_choose_method']}")
print("This will be applied to future job creations")
print(json.dumps(update_checkpoint_choosing, sort_keys=True, indent=4))

updated_job = tao_client.update_job(job_id=job_map["train_" + model_name], update_data=update_checkpoint_choosing)
print(json.dumps(updated_job, indent=4))

#### Push model to private ngc team registry

In [ ]:
# Publish model using TAO SDK
train_job_id = job_map["train_" + model_name]

try:
    publish_result = tao_client.publish_model(
        job_id=train_job_id,
        display_name=f"TAO {model_name}",
        description=f"Trained {model_name} model",
        team_name="tao"
    )
    
    print(" Model published successfully to NGC!")
    print(f"Job ID: {train_job_id}")
    print(f"Display Name: TAO {model_name}")
    print(f"Team: tao")
    print("\nPublish Response:")
    print(json.dumps(publish_result, indent=4))
    
except Exception as e:
    print(f" Failed to publish model: {str(e)}")
    print("Make sure the job completed successfully and you have appropriate permissions")

#### Remove model from private ngc team registry

In [ ]:
# # Remove published model using TAO SDK
# train_job_id = job_map["train_" + model_name]
# try:
#     remove_result = tao_client.remove_published_model(
#         job_id=train_job_id,
#         team="tao"
#     )
#     print(" Published model removed successfully!")
#     print(json.dumps(remove_result, indent=4))
# except Exception as e:
#     print(f" Failed to remove published model: {str(e)}")

### Export <a class="anchor" id="head-18"></a>

In [ ]:
# Get default export specs using TAO SDK
export_spec_response = tao_client.get_job_schema(action="export", network_arch=model_name)
export_specs = export_spec_response.get("default", {})
print(" Default export specifications:")
print(json.dumps(export_specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes
print(json.dumps(export_specs, sort_keys=True, indent=4))

In [ ]:
# Create export job using TAO SDK
# Create a new experiment job with export action and parent job reference
parent_job_id = job_map["train_" + model_name]
export_job_name = f"{model_name}_export_job"

# Create export experiment job using TAO SDK interface
export_job_id = tao_client.create_job(
    kind="experiment",
    name=export_job_name,
    network_arch=model_name,
    encryption_key=encode_key,
    workspace=workspace_id,
    action="export",
    specs=export_specs,  # Pass as dict, not JSON string
    parent_job_id=parent_job_id,
    base_experiment_ids=[selected_ptm_id] if selected_ptm_id else None,
    # platform_id="9af1aa90-8ea5-5a11-98d9-3879cd0da92c",  # Optional: Pick from get_gpu_types output
)

print(" Export job created successfully!")
print(f"Export Job ID: {export_job_id}")
print(f"Parent Job ID: {parent_job_id}")
print(f"Action: export")

job_map["export_" + model_name] = export_job_id
print("\nUpdated Job Map:")
print(json.dumps(job_map, indent=4))
%store job_map

In [ ]:
# Monitor export job status using TAO SDK
export_job_id = job_map["export_" + model_name]

while True:    
    clear_output(wait=True)
    
    try:
        job_status = tao_client.get_job_metadata(export_job_id)
        
        print(f" Export Job Status")
        print(f"Job ID: {export_job_id}")
        print(f"Status: {job_status.get('status', 'Unknown')}")
        print(f"Progress: {job_status.get('progress', 'N/A')}")
        
        # Show detailed status information
        print("\nDetailed Status:")
        print(json.dumps(job_status.get("job_details", {}), sort_keys=True, indent=4))
        
        current_status = job_status.get("status", "Unknown")
        
        if current_status == "Error":
            print(" Export job failed!")
            break
            
        if current_status in ["Done", "Completed"]:
            print(" Export job completed successfully!")
            break
            
        if current_status in ["Canceled", "Paused"]:
            print(f" Export job {current_status}")
            break
            
    except Exception as e:
        print(f" Error fetching export job status: {str(e)}")
        print("Job might still be starting up...")
        
    time.sleep(15)

### TAO inference <a class="anchor" id="head-20"></a>

- Run inference on a set of images using the .tlt model created at train step

In [ ]:
# Get default inference specs using TAO SDK
inference_spec_response = tao_client.get_job_schema(action="inference", network_arch=model_name)
tao_inference_specs = inference_spec_response.get("default", {})
print(" Default inference specifications:")
print(json.dumps(tao_inference_specs, sort_keys=True, indent=4))

In [ ]:
# Apply changes to specs if necessary
print(json.dumps(tao_inference_specs, sort_keys=True, indent=4))

In [ ]:
# Create inference job using TAO SDK
# Create a new experiment job with inference action and parent job reference
parent_job_id = job_map["train_" + model_name]
inference_job_name = f"{model_name}_inference_job"

# Create inference experiment job using TAO SDK interface
inference_job_id = tao_client.create_job(
    kind="experiment",
    name=inference_job_name,
    network_arch=model_name,
    encryption_key=encode_key,
    workspace=workspace_id,
    action="inference",
    specs=tao_inference_specs,  # Pass as dict, not JSON string
    parent_job_id=parent_job_id,
    base_experiment_ids=[selected_ptm_id] if selected_ptm_id else None,
    # platform_id="9af1aa90-8ea5-5a11-98d9-3879cd0da92c",  # Optional: Pick from get_gpu_types output
)

print(" Inference job created successfully!")
print(f"Inference Job ID: {inference_job_id}")
print(f"Parent Job ID: {parent_job_id}")
print(f"Action: inference")

job_map["inference_tao_" + model_name] = inference_job_id
print("\nUpdated Job Map:")
print(json.dumps(job_map, indent=4))
%store job_map

In [ ]:
# Monitor inference job status using TAO SDK
inference_job_id = job_map["inference_tao_" + model_name]

while True:    
    clear_output(wait=True)
    
    try:
        job_status = tao_client.get_job_metadata(inference_job_id)
        
        print(f" Inference Job Status")
        print(f"Job ID: {inference_job_id}")
        print(f"Status: {job_status.get('status', 'Unknown')}")
        print(f"Progress: {job_status.get('progress', 'N/A')}")
        
        # Show detailed status information
        print("\nDetailed Status:")
        print(json.dumps(job_status.get("job_details", {}), sort_keys=True, indent=4))
        
        current_status = job_status.get("status", "Unknown")
        
        if current_status == "Error":
            print(" Inference job failed!")
            break
            
        if current_status in ["Done", "Completed"]:
            print(" Inference job completed successfully!")
            break
            
        if current_status in ["Canceled", "Paused"]:
            print(f" Inference job {current_status}")
            break
            
    except Exception as e:
        print(f" Error fetching inference job status: {str(e)}")
        print("Job might still be starting up...")
        
    time.sleep(15)

### Delete Jobs<a class="anchor" id="head-22"></a>

In [ ]:
# Delete jobs instead of experiments
# Delete all created jobs using TAO SDK

print(" Deleting all created jobs...")

jobs_to_delete = []
for job_key, job_id in job_map.items():
    try:
        delete_result = tao_client.delete_job(job_id)
        print(f" Deleted job: {job_key} (ID: {job_id})")
    except Exception as e:
        print(f" Failed to delete job {job_key} (ID: {job_id}): {str(e)}")

print(f"\n Job cleanup completed! Processed {len(jobs_to_delete)} jobs.")

### Delete dataset <a class="anchor" id="head-23"></a>

#### Delete train dataset

In [ ]:
# Delete train dataset using TAO SDK
try:
    delete_result = tao_client.delete_dataset(train_dataset_id)
    print(" Train dataset deleted successfully!")
    print(f"Dataset ID: {train_dataset_id}")
    if delete_result:
        print("Delete Response:")
        print(json.dumps(delete_result, indent=4))
except Exception as e:
    print(f" Failed to delete train dataset {train_dataset_id}: {str(e)}")

#### Delete test dataset

In [ ]:
# Delete test dataset using TAO SDK
try:
    delete_result = tao_client.delete_dataset(test_dataset_id)
    print(" Test dataset deleted successfully!")
    print(f"Dataset ID: {test_dataset_id}")
    if delete_result:
        print("Delete Response:")
        print(json.dumps(delete_result, indent=4))
except Exception as e:
    print(f" Failed to delete test dataset {test_dataset_id}: {str(e)}")

#### Create backup file of Database to cloud for quicker turnaround time when re-deploying the API service
##### This is optional and can be skipped if you don't want to create a backup file

In [ ]:
# Create workspace backup using TAO SDK
try:
    backup_workspace_metadata = {
        "cloud_type": cloud_metadata["cloud_specific_details"]["cloud_type"],
        "cloud_specific_details": cloud_metadata["cloud_specific_details"]
    }
    
    backup_result = tao_client.backup_workspace(
        backup_file_name="mongodb_backup.gz",
        workspace_metadata=backup_workspace_metadata
    )
    print(" Workspace backup created successfully!")
    print(f"Backup file: mongodb_backup.gz")
    print("Backup Response:")
    print(json.dumps(backup_result, indent=4))
except Exception as e:
    print(f" Failed to create workspace backup: {str(e)}")